In [1]:
spark

In [46]:
import json
import mimeparse
import requests
import urllib
from pprint import pprint as pp

MEETUP_API_HOST = 'https://api.meetup.com'
EVENTS_URL = MEETUP_API_HOST + '/2/events.json'
MEMBERS_URL = MEETUP_API_HOST + '/2/members.json'
GROUPS_URL = MEETUP_API_HOST + '/2/groups.json'
RSVPS_URL = MEETUP_API_HOST + '/2/rsvps.json'
PHOTOS_URL = MEETUP_API_HOST + '/2/photos.json'
GROUP_URLNAME = 'London-Machine-Learning-Meetup'
# GROUP_URLNAME = 'London-Machine-Learning-Meetup' # 'Data-Science-London'

"""
Retrieves information about meetup.com
"""
def get_past_events():
    params = {
            'key': '7e27512d622b677838157d160347c78',
            'group_urlname': GROUP_URLNAME,
            'status': 'past',
            'desc': 'true'
    }
    
    query = urllib.parse.urlencode(params)
    url = '{0}?{1}'.format(EVENTS_URL, query)
    response = requests.get(url, timeout=1)
    data = response.json()['results']
    return data

"""
Get meetup members for a given meetup group
"""
def get_members():
    params = {
            'key': '7e27512d622b677838157d160347c78',
            'group_urlname': GROUP_URLNAME,
            'offset': '0',
            'format': 'json',
            'page': '100',
            'order': 'name'
    }
    
    query = urllib.parse.urlencode(params)
    url = '{0}?{1}'.format(MEMBERS_URL, query)
    response = requests.get(url, timeout=5)
    data = response.json()['results']
    return data
    
last_meetups = get_past_events()
#pp(last_meetups[2])

members = get_members()
members

[{'city': 'London',
  'country': 'gb',
  'id': 227321071,
  'joined': 1494432967000,
  'lat': 51.52,
  'link': 'http://www.meetup.com/members/227321071',
  'lon': -0.1,
  'name': ' J.C. Q.',
  'other_services': {},
  'photo': {'base_url': 'https://secure.meetupstatic.com',
   'highres_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/highres_266805991.jpeg',
   'photo_id': 266805991,
   'photo_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/member_266805991.jpeg',
   'thumb_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/thumb_266805991.jpeg',
   'type': 'member'},
  'self': {'common': {}},
  'state': '17',
  'status': 'active',
  'topics': [{'id': 9696, 'name': 'New Technology', 'urlkey': 'newtech'},
   {'id': 15018, 'name': 'Music', 'urlkey': 'music'},
   {'id': 8476, 'name': 'Education', 'urlkey': 'education'},
   {'id': 108403,
    'name': 'Technology Startups',
    'urlkey': 'technology-startups'},
   {'id': 7203, 'name': 'Education & Techno

In [54]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("SparkSQL").getOrCreate()

In [57]:
membersRDD = sc.parallelize(members)

[{'city': 'London',
  'country': 'gb',
  'id': 227321071,
  'joined': 1494432967000,
  'lat': 51.52,
  'link': 'http://www.meetup.com/members/227321071',
  'lon': -0.1,
  'name': ' J.C. Q.',
  'other_services': {},
  'photo': {'base_url': 'https://secure.meetupstatic.com',
   'highres_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/highres_266805991.jpeg',
   'photo_id': 266805991,
   'photo_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/member_266805991.jpeg',
   'thumb_link': 'https://secure.meetupstatic.com/photos/member/b/3/a/7/thumb_266805991.jpeg',
   'type': 'member'},
  'self': {'common': {}},
  'state': '17',
  'status': 'active',
  'topics': [{'id': 9696, 'name': 'New Technology', 'urlkey': 'newtech'},
   {'id': 15018, 'name': 'Music', 'urlkey': 'music'},
   {'id': 8476, 'name': 'Education', 'urlkey': 'education'},
   {'id': 108403,
    'name': 'Technology Startups',
    'urlkey': 'technology-startups'},
   {'id': 7203, 'name': 'Education & Techno

8

In [41]:
jsonRDD = sc.wholeTextFiles("c:/data/meetup.json",4).map(lambda x: x[1])
jsonRDD.getNumPartitions()

1

In [32]:
import re
js = jsonRDD.map(lambda x: re.sub(r"\s+", "", x, flags=re.UNICODE))
js

PythonRDD[55] at RDD at PythonRDD.scala:48

In [34]:

meetup= sqlContext.read.json(js)

In [35]:
meetup.printSchema()

root
 |-- bio: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- hometown: string (nullable = true)
 |-- id: long (nullable = true)
 |-- joined: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- link: string (nullable = true)
 |-- lon: double (nullable = true)
 |-- name: string (nullable = true)
 |-- other_services: struct (nullable = true)
 |    |-- facebook: struct (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |    |-- flickr: struct (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |    |-- linkedin: struct (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |    |-- tumblr: struct (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |    |-- twitter: struct (nullable = true)
 |    |    |-- identifier: string (nullable = true)
 |-- photo: struct (nullable = true)
 |    |-- base_url: string (nullable = true)
 |    |-- highres_link: str